In [1]:
import warnings
from loguru import logger

warnings.simplefilter(action='ignore', category=FutureWarning)

from edgedroid.models.timings import *
import edgedroid.data as default_data

logger.enable("edgedroid")

data = default_data.load_curve_fitting_data()
data

In [2]:
from collections import deque
import pandas as pd
from typing import Tuple, Deque
from tqdm.notebook import tqdm
import numpy as np

result_rows = deque()
for participant in tqdm(data["participant"].unique(), desc="Run"):
    class MultiCurveFittingTestModel(MultiCurveFittingExecutionTimeModel):
        @staticmethod
        def get_data() -> pd.DataFrame:
            data = MultiCurveFittingExecutionTimeModel.get_data()
            return data[data["participant"] != participant].copy()
        
    class PowerFitTestModel(MultiCurveFittingTestModel):
        _fit_functions = (PowerFit,)
        
    class SquareFitTestModel(MultiCurveFittingTestModel):
        _fit_functions = (SquareFit,)
        
    class CubeFitTestModel(MultiCurveFittingTestModel):
        _fit_functions = (CubeFit,)
        
    class ExponentialFitTestModel(MultiCurveFittingTestModel):
        _fit_functions = (ExponentialFit,)
        
    def get_test_data(*args, **kwargs) -> (
        Tuple[
            pd.DataFrame,
            pd.arrays.IntervalArray,
            pd.arrays.IntervalArray,
            pd.arrays.IntervalArray,
        ]
    ):
        data, *rest = ExecutionTimeModel.get_data()
        return (data[data["run_id"] != participant].copy(), *rest)

    class EmpiricalTestModel(EmpiricalETM):
        get_data = get_test_data
        
    class EmpiricalMeanTestModel(EmpiricalAggregateETM):
        get_data = get_test_data
        
        def __init__(self, *args, **kwargs):
            super().__init__(*args, aggregate_fn=np.mean, **kwargs)
            
    class EmpiricalMedianTestModel(EmpiricalAggregateETM):
        get_data = get_test_data
        
        def __init__(self, *args, **kwargs):
            super().__init__(*args, aggregate_fn=np.median, **kwargs)
        
    class TheoreticalTestModel(FittedETM):
        get_data = get_test_data

    class NaiveTestModel(FirstOrderETM):
        get_data = get_test_data
        
    class NaiveAggTestModel(FirstOrderAggregateETM):
        get_data = get_test_data
        
    cleanup = CleanupMode.TRUNCATE
    
    run_data = data[data["participant"] == participant].copy()

    neuroticism = run_data["neuroticism"].unique()[0]
    naive_model = NaiveTestModel()
    naive_mean_model = NaiveAggTestModel(np.mean)
    naive_median_model = NaiveAggTestModel(np.median)
    # curve_fitting_model = CurveFittingTestModel(neuroticism)
    
    power_fit_model = PowerFitTestModel(neuroticism)
    square_fit_model = SquareFitTestModel(neuroticism)
    cube_fit_model = CubeFitTestModel(neuroticism)
    exponential_fit_model = ExponentialFitTestModel(neuroticism)
    multi_fit_model = MultiCurveFittingExecutionTimeModel(neuroticism)
    
    # deque containing (model name, model object, ttf_bins, window_size, kernel name)
    models: Deque[Tuple[str, ExecutionTimeModel, int, int, str]] = deque()
    models.extend((
        ("1st order", naive_model, 0, 0, "none"),
        ("1st order (median)", naive_median_model, 0, 0, "none"),
        ("1st order (mean)", naive_mean_model, 0, 0, "none"),
        ("a * x^b + c", power_fit_model , 0, 0, "none"),
        ("a * x^2 + b * x + c", square_fit_model, 0, 0, "none"),
        ("a * x^3 + b * x^2 + c * x + d", cube_fit_model, 0, 0, "none"),
        ("a * e^x + b", exponential_fit_model, 0, 0, "none"),
        ("multi curve", multi_fit_model, 0, 0, "none")
    ))
    
    # for (ttf_bins, window_size) in itertools.product(range(1, 10), (4, 8, 12)):
    #     exp_kernel = ExponentialTTFWindowKernel(window_size=window_size)
    #     steep_linear_kernel = LinearTTFWindowKernel(window_size=window_size, max_relative_weight=10)
    #     shallow_linear_kernel = LinearTTFWindowKernel(window_size=window_size, max_relative_weight=2)
    #     average_kernel = AverageTTFWindowKernel(window_size=window_size)
    #     
    #     kernels = (
    #         ("exponential", exp_kernel),
    #         ("linear_steep", steep_linear_kernel),
    #         ("linear_shallow", shallow_linear_kernel),
    #         ("average", average_kernel)
    #     )
    #     model_classes = (
    #         ("empirical", EmpiricalTestModel),
    #         ("empirical (mean)", EmpiricalMeanTestModel),
    #         ("empirical (median)", EmpiricalMedianTestModel),
    #         ("theoretical", TheoreticalTestModel)
    #     )
    #     
    #     for (class_name, model_cls), (kernel_name, kernel) in itertools.product(model_classes, kernels):
    #         m = model_cls(kernel=kernel, neuroticism=neuroticism, ttf_levels=ttf_bins, cleanup=cleanup)
    #         models.append((class_name, m, ttf_bins, window_size, kernel_name))
        
    for _ in range(1):
        for i, (prev_ttf, exec_time, neuro, prev_duration) in enumerate(run_data[["prev_ttf", "exec_time", "neuro", "prev_duration"]].itertuples(index=False)):
            for model_name, model, ttf_bins, window_size, kernel_name in models:
                if i == 0:
                    # reset the model between runs
                    model.reset()
                else:
                    # first row has no previous ttf
                    model.advance(prev_ttf)
                    
                prediction = model.get_execution_time()
                error = prediction - exec_time
                result_rows.append({
                    "real": exec_time,
                    "prediction": prediction,
                    "error": error,
                    "sqr_error": np.square(error),
                    "model": model_name,
                    "duration": prev_duration,
                    "neuro": neuro,
                    # "kernel": kernel_name,
                    # "ttf_bins": ttf_bins,
                    # "window_size": window_size,
                })

results = pd.DataFrame(result_rows)
for col in ("model", ):  # "kernel"):
    results[col] = results[col].astype("category")

results.to_parquet("./full_validation.parquet")
results

In [3]:
# results.groupby(["model", "kernel", "window_size", "ttf_bins"], observed=True)["sqr_error"].mean().sort_values(ascending=True)

results.groupby(["model", "neuro", "duration"], observed=True)["sqr_error"].mean().sort_values(ascending=True)

# look at neuroticism
# look at durations
# look different functions
# look at delta in ttf to reset duration (maybe not instantaneous delta)

# look at different curves for each quadrant

In [4]:
results.groupby(["model"], observed=True)["sqr_error"].describe().sort_values(by="mean", ascending=True)


In [5]:
# # %matplotlib inline
# import matplotlib.pyplot as plt
# import seaborn as sns
# 
# fg = sns.catplot(results, kind="point", x="ttf_bins", y="sqr_error", hue="model_tag", height=5, aspect=2)
# fg.set(ylim=(0, None))
# fg.set_ylabels("MSE")
# for ax in fg.axes.flat:
#     ax.set_xticks(ax.get_xticks(), ax.get_xticklabels(), rotation=45, ha='right')
# plt.show()

In [6]:
# results.groupby(["model", "ttf_bins", "cleanup"])["sqr_error"].mean().sort_values()